In [1]:
import glob
import os
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    if len(sentence.split()) > 15: 
        sentence = ""
    
    return sentence

In [3]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    #if len(sentence.split()) >= 12: continue    

    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence) == 0: continue    
    corpus.append(preprocess_sentence(sentence))

# 정제된 결과 10개 확인
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [4]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성한다.
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환한다.
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춘다.
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춘다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

In [5]:
tensor, tokenizer = tokenize(corpus)

src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input,
                                                          random_state=32,
                                                          test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe7886301c0>
Source Train: (124810, 14)
Target Train: (124810, 14)


In [6]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함해 7001개
VOCAB_SIZE = tokenizer.num_words + 1

train_set = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
train_set = train_set.batch(BATCH_SIZE, drop_remainder = True)
test_set = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
test_set = test_set.batch(BATCH_SIZE, drop_remainder = True)

In [10]:
class TextGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_size, hidden_size):
    super(TextGenerator, self).__init__()

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
    self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
    self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
    self.linear = tf.keras.layers.Dense(vocab_size)

  def call(self, x):
    out = self.embedding(x)
    out = self.rnn_1(out)
    out = self.rnn_2(out)
    out = self.linear(out)

    return out

embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [11]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(train_set, validation_data = test_set, epochs=10)

Epoch 1/10
487/487 [==============================] - 254s 484ms/step - loss: 3.2830 - val_loss: 2.9067
Epoch 2/10
487/487 [==============================] - 236s 484ms/step - loss: 2.7343 - val_loss: 2.6526
Epoch 3/10
487/487 [==============================] - 236s 485ms/step - loss: 2.4161 - val_loss: 2.4703
Epoch 4/10
487/487 [==============================] - 236s 484ms/step - loss: 2.1079 - val_loss: 2.3309
Epoch 5/10
487/487 [==============================] - 236s 484ms/step - loss: 1.8140 - val_loss: 2.2333
Epoch 6/10
487/487 [==============================] - 236s 485ms/step - loss: 1.5547 - val_loss: 2.1684
Epoch 7/10
487/487 [==============================] - 236s 485ms/step - loss: 1.3446 - val_loss: 2.1395
Epoch 8/10
487/487 [==============================] - 236s 485ms/step - loss: 1.1872 - val_loss: 2.1368
Epoch 9/10
487/487 [==============================] - 236s 485ms/step - loss: 1.0844 - val_loss: 2.1481
Epoch 10/10
487/487 [==============================] - 236s 485m

In [12]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        if predict_word.numpy()[0] == end_token:
            break
        if test_tensor.shape[1] >= max_len:
            break
    
    generated = ""
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    
    return generated

In [13]:
generate_text(model, tokenizer, init_sentence="<start> sunday", max_len=20)

'<start> sunday , you ain t go to church though <end> '